# Introduction to the Monte Carlo method

----

Start by defining the [Gibbs (or Boltzmann) distribution](https://en.wikipedia.org/wiki/Boltzmann_distribution):
$$P(\alpha) = e^{-E(\alpha)/kT}$$
this expression, defines the probability of observing a particular configuration of spins, $\alpha$. 
As you can see, the probability of $\alpha$ decays exponentially with increasing energy of $\alpha$, $E(\alpha)$,
where $k$ is the Boltzmann constant, $k = 1.38064852 \times 10^{-23} J/K$
and $T$ is the temperature in Kelvin. 

## What defines the energy of a configuration of spins? 
Given a configuration of spins (e.g., $\uparrow\downarrow\downarrow\uparrow\downarrow$) we can define the energy using what is referred to as an Ising Hamiltonian:
$$ \hat{H}' = \frac{\hat{H}}{k} = -\frac{J}{k}\sum_{<ij>} s_is_j,$$
where, $s_i=1$ if the $i^{th}$ spin is `up` and $s_i=-1$ if it is `down`, and the brackets $<ij>$ indicate a sum over spins that are connected,
and $J$ is a constant that determines the energy scale. 
The energy here has been divided by the Boltzmann constant to yield units of temperature. 
Let's consider the following case, which has the sites connected in a single 1D line:
$$\alpha = \uparrow-\downarrow-\downarrow-\uparrow-\downarrow.$$ 
What is the energy of such a configuration?
$$ E(\alpha)' = J/k(-1 + 1 - 1 - 1) = \frac{E(\alpha)}{k} = -2J/k$$

## P1: Write a class that defines a spin configuration

In [2]:
class spin_config:
    def __init__(self):
        self.config = []
        self.N = 0
    def n_sites(self):
        return len(self.config)
    def initialize(self,list):
        for i in list:
            self.config.append(i)
    def reset(self):
        self.config.clear()
    def configuration(self):
        return self.config
        

## P2: Write a class that defines the 1D hamiltonian, containing a function that computes the energy of a configuration

In [3]:
import math
k=1.38064852*math.pow(10,-23)
k=1
J=-2
mu=1.1
class base_energy:
    def __init__(self):
        self.energy = 0
        self.state = []
    def initialize(self,list):
        for i in list:
            self.state.append(i)
    def reset(self):
        self.state.clear()
        self.energy = 0
    def spin_energy(self):
        for i in range(len(self.state)-1):
            self.energy+=(self.state[i]*self.state[i+1])
        self.energy*= J/k
    
        for j in self.state:
            self.energy+=(mu*j)
            
        return self.energy        

## Q3: What is the energy for (++-+---+--+)?

In [4]:
lattice_energy = base_energy()
list = [1,1,-1,1,-1,-1,-1,1,-1,-1,1]
lattice_energy.initialize(list)
lattice_energy.spin_energy()

2.8999999999999995

## Properties
For any fixed state, $\alpha$, the `magnetization` ($M$) is proportional to the _excess_ number of spins pointing up or down while the energy is given by the
Hamiltonian:
$$M(\alpha) = N_{\text{up}}(\alpha) - N_{\text{down}}(\alpha).$$
As a dynamical, fluctuating system, each time you measure the magnetization, the system might be in a different state ($\alpha$) and so you'll get a different number!
However, we already know what the probability of measuring any particular $\alpha$ is, so in order to compute the average magnetization, $\left<M\right>$, we just need to multiply the magnetization of each possible configuration times the probability of it being measured, and then add them all up!
$$ \left<M\right> = \sum_\alpha M(\alpha)P(\alpha).$$
In fact, any average value can be obtained by adding up the value of an individual configuration multiplied by it's probability:
$$ \left<E\right> = \sum_\alpha E(\alpha)P(\alpha).$$

This means that to obtain any average value (also known as an `expectation value`) computationally, we must compute the both the value and probability of all possible configurations. This becomes extremely expensive as the number of spins ($N$) increases. 

## P3: Write a function that computes the magnetization of a spin configuration

In [5]:
def magnetization(spin_config):
    magnet = 0
    for i in spin_config:
        if i==1:
            magnet+=1
        else:
            magnet-=1

    return magnet

## Average Energy, Heat Capacity, and Magnetic Susceptibility

In [1]:
import math
def split(word):
    return [char for char in word]


class average_energy:
    def __init__(self):
        self.energies = []
        self.magnetizations = []
        self.probabilities = []
        self.states = []
        self.k = 1
        self.j = -2
        self.mu = 0
        self.T = 323
        self.avg_eng = 0
        self.avg_mag = 0
        self.heat_capacity = 0
        self.mag_sus = 0
    def reset(self):
        self.energies = []
        self.magnetizations = []
        self.probabilities = []
        self.states = []
        self.k = 1
        self.j = -2
        self.mu = 0
        self.T = 323
        self.avg_eng = 0
        self.avg_mag = 0
        self.heat_capacity = 0
        self.mag_sus = 0
    def constants(self,boltz,pref,mu,temp):
        self.k = boltz
        self.j = pref
        self.mu = mu
        self.T = temp    
    def generate_state(self,n):
        for i in range(2**n):
            binary = bin(i)
            state = split(binary)
            state.remove('0')
            state.remove('b')
            for j in range(len(state)):
                state[j] = int(state[j])
                if state[j]==0:
                    state[j]=-1
            while len(state)<n:
                state.insert(0,-1)
            self.states.append(state)
        
        for j in self.states:
            energy = 0
            for i in range(len(j)-1):
                energy+=(j[i]*j[i+1])
            energy *= self.j/self.k
            
            for k in j:
                energy+=(self.mu*k)
            self.energies.append(energy)
            
        for j in self.states:
            magnet = 0
            for i in j:
                magnet+=i
            self.magnetizations.append(magnet)
        
        for i in self.energies:
            self.probabilities.append(math.pow(math.e,-i/(self.k*self.T)))
        normalization_factor = sum(self.probabilities)
        for j in range(len(self.probabilities)):
            self.probabilities[j]/=normalization_factor
        
        for i in range(len(self.energies)):
            self.avg_eng+=self.energies[i]*self.probabilities[i] 
        
        for i in range(len(self.magnetizations)):
            self.avg_mag+=self.magnetizations[i]*self.probabilities[i]
        copy = []
        E = self.avg_eng
        for i in range(len(self.energies)):
            copy.append(self.energies[i])
        
        for i in range(len(self.energies)):
            self.energies[i] = math.pow(self.energies[i],2)
        
        self.avg_eng = 0
        for i in range(len(self.energies)):
            self.avg_eng+=self.energies[i]*self.probabilities[i]
            
        self.heat_capacity = (self.avg_eng - E)/(self.k*math.pow(self.T,2))
        
        self.energies = copy
        
        self.avg_eng = 0
        for i in range(len(self.energies)):
            self.avg_eng+=self.energies[i]*self.probabilities[i]
            
        copy = []
        E = self.avg_mag
        for i in range(len(self.magnetizations)):
            copy.append(self.magnetizations[i])
        
        for i in range(len(self.magnetizations)):
            self.magnetizations[i] = math.pow(self.magnetizations[i],2)
        
        self.avg_mag = 0
        for i in range(len(self.magnetizations)):
            self.avg_mag+=self.magnetizations[i]*self.probabilities[i]
            
        self.mag_sus = (self.avg_eng - E)/(self.k*math.pow(self.T,2))
        
        self.magnetizations = copy
        
        self.avg_mag = 0
        for i in range(len(self.magnetizations)):
            self.avg_mag+=self.magnetizations[i]*self.probabilities[i]
            
        
        
tester = average_energy()
tester.constants(1,-1,.1,323)
tester.generate_state(5)

## Q2: How many configurations are possible for:

(a) N=10?

$2^{10}$

(b) N=100?

$2^{100}$

(c) N=1000?

$2^{1000}$